### Imports

In [ ]:
# Package imports
import os, subprocess
from itertools import combinations
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
# import libraries for biological data analysis
from coolpuppy import coolpup
from coolpuppy.lib import numutils
from coolpuppy.lib.puputils import divide_pups
from coolpuppy import plotpup
import cooler
import bioframe
import cooltools
from cooltools import expected_cis, expected_trans
from cooltools.lib import plotting
import cooler as clr
import h5py
# import open2c libraries
import bioframe
from itertools import product
from scipy.stats import pearsonr

# P(s) curve plotting

In [ ]:
file_path = "" # path to mcool files
mcool_files = [""] # list of mcool files to analyze

resolution = 1000 # mcool resolutions
all_coolers = [read_cooler(os.path.join(file_path, mcool_file), resolution) for mcool_file in mcool_files]

# Use bioframe to fetch the genomic features from the UCSC.
mm39_chromsizes = bioframe.fetch_chromsizes('mm39')
mm39_cens = bioframe.fetch_centromeres('mm39')
# create a view with chromosome arms using chromosome sizes and definition of centromeres
mm39_arms = bioframe.make_chromarms(mm39_chromsizes,  mm39_cens)

# select only those chromosomes available in cooler
mm39_arms = mm39_arms[mm39_arms.chrom.isin(all_coolers[0].chromnames)].reset_index(drop=True)

### Contact vs distance calculation

In [ ]:
# cvd == contacts-vs-distance
all_cvds = []
cvd_path = "" # output path for contact vs distance calculations
for all_cooler, mcool_file in zip(all_coolers, mcool_files):
    cvd = cooltools.expected_cis(
        clr=all_cooler,
        view_df=mm39_arms,
        smooth=True,
        aggregate_smoothed=True,
        nproc=20 #if you do not have multiple cores available, set to 1
    )
    name = mcool_file.split(".")[0] + ".csv"
    cvd.to_csv(os.path.join(cvd_path, name))
    all_cvds.append(cvd)

### Plot contact vs distance

In [ ]:
pruned_cvds = [cvd[~cvd["region1"].isin(['chrY_p', 'chrM_p'])] for cvd in all_cvds] # remove Y and M chromosome from analysis

f, ax = plt.subplots(2,1,gridspec_kw={'height_ratios':[6,2]},
    sharex=True, figsize=(6, 8))

labels=[""] # condition labels

for i, cvd in enumerate(pruned_cvds):
    cvd['s_bp'] = cvd['dist']* resolution
    cvd['balanced.avg.smoothed.agg'].loc[cvd['dist'] < 2] = np.nan
    ax[0].loglog(
        cvd['s_bp'],
        cvd['balanced.avg.smoothed.agg'],
    )
    ax[0].set(
        ylabel='IC contact frequency')

    ax[0].grid(lw=0.5)
    ax[0].set_title("") # set the plot title
    ax[0].set_xticklabels([])

    der = np.gradient(np.log(cvd['balanced.avg.smoothed.agg']),
      np.log(cvd['s_bp']))
    ax[1].semilogx(
        cvd['s_bp'],
        der,
        alpha=0.5
    )
    ax[1].set_ylabel('slope')
    ax[1].set(xlabel='separation, bp')
    ax[1].grid(lw=0.5)
    
ax[0].legend(labels)
plot_dir = "" # output directory for plots
plt.savefig(os.path.join(plot_dir, "name_of_plot.svg"), format='svg')

# Compartment calls

In [ ]:
## fasta sequence is required for calculating binned profile of GC conent
if not os.path.isfile('./mm39.fa'):
    ## note downloading a ~1Gb file can take a minute
    subprocess.call('wget --progress=bar:force:noscroll https://hgdownload.cse.ucsc.edu/goldenpath/mm39/bigZips/mm39.fa.gz', shell=True)
    subprocess.call('gunzip mm39.fa.gz', shell=True)

In [ ]:
# read mcools
file_path = "" # path to mcool files
mcool_files = [""] # list of mcool files

resolution = 500000
all_coolers = [read_cooler(os.path.join(file_path, mcool_file), resolution) for mcool_file in mcool_files]

In [1]:
# make view_df
bins = all_coolers[0].bins()[:]
mm39_genome = bioframe.load_fasta('./mm39.fa');
## note the next command may require installing pysam
gc_cov = bioframe.frac_gc(bins[['chrom', 'start', 'end']], mm39_genome)
gc_cov.to_csv('mm39_gc_cov_100kb.tsv',index=False,sep='\t')
display(gc_cov)

view_df = pd.DataFrame({'chrom': all_coolers[0].chromnames,
                        'start': 0,
                        'end': all_coolers[0].chromsizes.values,
                        'name': all_coolers[0].chromnames}
                      )
view_df = view_df[view_df.chrom != 'chrY']
view_df = view_df[view_df.chrom != 'chrM']

display(view_df)

### Compute eigenvectors

In [ ]:
comp_path = "" # output path for eigenvector computation
all_eigenvals, all_eigenvectors = [], []
for all_cooler, mcool_name in zip(all_coolers, mcool_files):
    # obtain first 3 eigenvectors
    eigvals, eigvecs = cooltools.eigs_cis(
                            all_cooler,
                            gc_cov,
                            view_df=view_df,
                            n_eigs=3,
                            )
    name = mcool_name.split(".")[0]
#     eigvals.to_csv(os.path.join(comp_path, name + "_eigvals.csv"))
#     eigvecs.to_csv(os.path.join(comp_path, name + "_eigvecs.csv"))
    all_eigenvals.append(eigvals)
    all_eigenvectors.append(eigvecs)

### Plot eigenvector

In [ ]:
unt_cis_eigs = all_eigenvectors[0]
unt_eigenvector_track = unt_cis_eigs[['chrom','start','end','E1']]
f, ax = plt.subplots(
    figsize=(15, 10),
)

chr_start_mb = 5
chr_end_mb = 190

norm = LogNorm(vmax=0.1)

im = ax.matshow(
    all_coolers[0].matrix()[:],
    norm=norm,
    cmap='fall'
);
plt.axis([chr_start_mb *2,chr_end_mb *2,chr_end_mb *2,chr_start_mb *2])

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im, cax=cax, label='corrected frequencies');
ax.set_ylabel(f'chr1:{chr_start_mb}-{chr_end_mb}Mb')
ax.xaxis.set_visible(False)

ax1 = divider.append_axes("top", size="20%", pad=0.25, sharex=ax)
weights = all_coolers[0].bins()[:]['weight'].values
ax1.plot([chr_start_mb*10,chr_end_mb*10],[0,0],'k',lw=0.25)
norm_e1 = unt_eigenvector_track['E1'].values #/ np.amax(unt_eigenvector_track['E1'].values)
ax1.fill_between(np.arange(0, len(norm_e1)), 0, norm_e1, where=norm_e1>0, facecolor='red', interpolate=True)
ax1.fill_between(np.arange(0, len(norm_e1)), 0, norm_e1, where=norm_e1<=0, facecolor='green', interpolate=True)

ax1.plot( unt_eigenvector_track['E1'].values / np.amax(unt_eigenvector_track['E1'].values), label='E1')
ax1.set_ylim([np.nanmin(unt_eigenvector_track['E1'].values), np.nanmax(unt_eigenvector_track['E1'].values)])

ax1.set_xticks([])
ax1.set_yticks([])

plot_dir = "" # output directory
plt.savefig(os.path.join(plot_dir, "name.svg"), format='svg')

### Plot correlation matrix

In [ ]:
eigenvector_len = len(all_eigenvectors)
fig, ax = plt.subplots(eigenvector_len, eigenvector_len, figsize=(12,12), dpi=600,
                      sharex=True, sharey=True)

labels=[""] # 
for i, j in product(range(eigenvector_len), range(eigenvector_len)):
    if i <= j:
        eigenvector_1, eigenvector_2 = all_eigenvectors[i]['E1'], all_eigenvectors[j]['E1']
        nan_mask = np.logical_and(~np.isnan(eigenvector_1), ~np.isnan(eigenvector_2))
        eigenvector_1 = eigenvector_1[nan_mask].to_numpy()
        eigenvector_2 = eigenvector_2[nan_mask].to_numpy()
        
        #subsample

        sns.kdeplot(x=eigenvector_1, 
                    y=eigenvector_2 + 1e-6 * np.random.standard_normal(eigenvector_2.shape), 
                    ax=ax[i, j], thresh=0.05, fill=True, levels=8)
        corr_coefficient = pearsonr(eigenvector_1, 
                                    eigenvector_2).statistic
        ax[i, j].text(0.02, 0.9,
                 f'{corr_coefficient:.2f}',
                 ha='left',
                 va='top',
                 transform=ax[i, j].transAxes)
        if i == 0:
            ax[i, j].set_title(labels[j])
        
        if i == j:
            ax[i, j].set_ylabel(labels[i])
        
        ax[i, j].set_xlim([-1.5, 1.5])
        ax[i, j].set_ylim([-1.5, 1.5])
        ax[i, j].plot(np.linspace(-2, 2), np.linspace(-2, 2), 'k', linewidth=0.5)
    else:
        fig.delaxes(ax[i][j])
        
plot_dir = "" # output directory
plt.savefig(plot_dir, format='svg')